In [1]:
import datetime
import os
import paramiko
import pandas as pd

In [2]:
def recrusive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
remote_server_root="/mnt/drv5/saatva/BL_data_for_Circ_analysis/"

In [4]:
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)




In [5]:
daily_data=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_data=[x for x in daily_data if ("Daily" in x) & ("2019" in x)]
df_daily_data=pd.DataFrame({"file_path":daily_data})

df_daily_data["week_end_date"]=df_daily_data["file_path"].apply(lambda x: x.split("ts/2019_by_weeks/MediaStorm_")[1][:10])
df_daily_data=df_daily_data[df_daily_data['week_end_date']>"2019-04-25"]
df_daily_data

,file_path,week_end_date
16,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-04-27
17,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-04


In [6]:
for local_path in df_daily_data['file_path'].tolist():
    remote_path="/mnt/drv5/saatva/BL_data_for_Circ_analysis/ItemLevel_2019/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)

In [7]:
sftp.close()